### Cloud partition algorithm for CM1 output
A cloud partition algorithm adapted from [Tulich et al. (2007)](https://journals.ametsoc.org/view/journals/atsc/64/4/jas3884.1.xml).<br>
Date: Nov. 2021

In [1]:
import sys
import os
import numpy as np
import xarray as xr
from glob import glob
import matplotlib.pyplot as plt
import warnings
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Rectangle
import metpy.calc as mpc
from metpy.units import units
# for animations
from PIL import Image

In [2]:
warnings.filterwarnings('ignore')

In [3]:
# aggregation indices
os.chdir('/data2/willytsai/python_module')
import conorgidx_revise as agi
from SCAI_calc4obj import labeled_obj, SCAI_calc
from COP_calc4obj import COP
from MICA_calc4obj import MICA

In [4]:
def running_mean(y,window_N):
    y_avg = np.zeros(len(y))
    avg_mask = np.ones(window_N) / window_N

    y_avg = np.convolve(y, avg_mask, 'same')
    y_avg[-window_N:] = y[-window_N:]
    y_avg[:window_N] = y[:window_N]
    return y_avg    

In [5]:
def read_data(data_dir,t_start=-360):
    os.chdir(data_dir)
    print(data_dir)
    file_name = glob('*nc')
    data_zon = xr.open_dataset('cm1out_zonmean.nc')
    data_3d = xr.open_dataset('cm1out_sub3d.nc')
    data_2d = xr.open_dataset('cm1out_2Dvars.nc')
    data_q3d = xr.open_dataset('cm1out_q3d.nc')
    
    #3d data
    th3d = data_3d.th[t_start:,:45,:,:]
    v3d = data_3d.vinterp[t_start:,:45,:,:]
    u3d = data_3d.uinterp[t_start:,:45,:,:]
    w3d = data_3d.winterp[t_start:,:45,:,:]
    qc3d = data_q3d.qc[t_start:,:45,:,:]
    qi3d = data_q3d.qi[t_start:,:45,:,:]
    qr3d = data_q3d.qr[t_start:,:45,:,:]
    qg3d = data_q3d.qg[t_start:,:45,:,:]
    qs3d = data_q3d.qs[t_start:,:45,:,:]
    
    rnp3d = (qc3d+qi3d)
    rp3d = (qr3d+qg3d+qs3d)
    
    # zonal mean data
    th = data_zon.th[t_start:,:,:,:]
    qv = data_zon.qv[t_start:,:,:,:]
#     u = data_zon.uinterp[t_start:t_end,:,:,:]
#     v = data_zon.vinterp[t_start:t_end,:,:,:]
#     w = data_zon.winterp[t_start:t_end,:,:,:]
    qc = data_zon.qc[t_start:,:,:,:]
    qi = data_zon.qi[t_start:,:,:,:]
    prs = data_zon.prs[t_start:,:,:,:]
    # 2d data
    prate = data_2d.prate[t_start:,:,:]
    pwat = data_2d.pwat[t_start:,:,:]
    cwp = data_2d.cwp[t_start:,:,:]
    cape = data_2d.cape[t_start:,:,:]
    cin = data_2d.cin[t_start:,:,:]
    
    # temperature conversion
    T = th/((1000*100/prs)**(287.15/1004))-273.15 # [degC]
    # density 
    ro = prs/(287.15*(T+273.15))

    x_dim = data_2d.lon
    y_dim = data_2d.lat
#    div = np.gradient(v,y_dim*1000,axis=2) # du/dx+dv/dy 
#    vor = -np.gradient(u,y_dim*1000,axis=2) # -du/dy+dv/dx
    #relative humidty calculation, Buck (1996)
    es_liq = 0.61121*np.exp((18.678-T/234.5)*(T/(257.14+T)))*10 #[hpa]
    es_ice = 0.61115*np.exp((23.036-T/333.7)*(T/(279.82+T)))*10

    qs_liq = 0.622*es_liq/(prs/100-es_liq)
    qs_ice = 0.622*es_ice/(prs/100-es_ice)

    rh = qv/qs_liq
    rh_ice = qv/qs_ice

    rh = rh.values.flatten()
    rh_ice = rh_ice.values.flatten()
    T_test = T.values.flatten()

    rh[T_test<0] = rh_ice[T_test<0]
    rh = rh.reshape((T.shape[0],T.shape[1],T.shape[2],T.shape[3]))
    rh = xr.DataArray(rh,coords=[T.time,T.lev,T.lat,T.lon],dims=['time','lev','lat','lon'])
    
#    return (th,T,qv,qc,qi,rh,prate*3600*24,pwat*1000,cwp,u,v,w,div,vor,cape,cin,prs,ro,th3d,v3d,u3d)
    return (th,T,qv,qc,qi,rh,prate*3600*24,pwat*1000,cwp,u3d,v3d,w3d,cape,cin,prs,ro,th3d,rp3d,rnp3d)

In [6]:
# get weather or weather2 
server = '/data2/willytsai/cm1r19.8/analysis/runs_cheyenne/'
exp_name = ['CM1_RAD4K_RCEP_240x240_lowU15_UNI']
#exp_short = ['LOW','MID','UPP','LOW_INT','LOW_UNI','MID_UNI','UPP_UNI']

os.chdir(server)
os.system('mkdir -p fig_sum') # create a figure archive
fig_dir = server+'fig_sum/' 

In [7]:
(th1,T1,qv1,qc1,qi1,rh1,prate1,pwat1,cwp1,u3d1,v3d1,w3d1,cape1,cin1,prs1,ro1,th3d1,rp3d1,rnp3d1) = read_data(server+exp_name[0])
#(th2,T2,qv2,qc2,qi2,rh2,prate2,pwat2,cwp2,u3d2,v3d2,w3d2,cape2,cin2,prs2,ro2,th3d2) = read_data(server+exp_name[1],t_start=360)
#(th3,T3,qv3,qc3,qi3,rh3,prate3,pwat3,cwp3,u3d3,v3d3,w3d3,cape3,cin3,prs3,ro3,th3d3) = read_data(server+exp_name[2],t_start=360)
#(th4,T4,qv4,qc4,qi4,rh4,prate4,pwat4,cwp4,u3d4,v3d4,w3d4,cape4,cin4,prs4,ro4,th3d4) = read_data(server+exp_name[3],t_start=360)
#(th5,T5,qv5,qc5,qi5,rh5,prate5,pwat5,cwp5,u3d5,v3d5,w3d5,cape5,cin5,prs5,ro5,th3d5) = read_data(server+exp_name[4],t_start=360)
#(th6,T6,qv6,qc6,qi6,rh6,prate6,pwat6,cwp6,u3d6,v3d6,w3d6,cape6,cin6,prs6,ro6,th3d6) = read_data(server+exp_name[5],t_start=360)
#(th7,T7,qv7,qc7,qi7,rh7,prate7,pwat7,cwp7,u3d7,v3d7,w3d7,cape7,cin7,prs7,ro7,th3d7) = read_data(server+exp_name[6],t_start=360)

/data2/willytsai/cm1r19.8/analysis/runs_cheyenne/CM1_RAD4K_RCEP_240x240_lowU15_UNI


In [8]:
# use the last 360 timesteps
prate1 = prate1[-360:,:]
pwat1 = pwat1[-360:,:]
ro1 = ro1[-360:,:]
w3d1 = w3d1[-360:,:]
rp3d1 = rp3d1[-360:,:]
rnp3d1 = rnp3d1[-360:,:]

In [9]:
x_dim = th3d1.lon
y_dim = th3d1.lat
z_dim = th3d1.lev

### Main codes 

In [10]:
from multiprocessing import Pool

In [11]:
def partition_multiprocess(i):
    "subfunction in cloudpartiion_2d: multiprocessing for each 1D column (index i)"

    rc_1d = rc2d[:,i]*1000 # g/kg
    rnp_1d = rnp2d[:,i]*1000 # g/kg
    idx_ct = np.where(rc_1d>0.1)[0]
    # if no > 0.1 g/kg
    if len(idx_ct) != 0:
        Zct = z_dim[idx_ct[-1]]
        idx_cb = np.where(rnp_1d<0.01)[0]
        Zcb = z_dim[idx_cb[0]]
    else:
        idx_ct = np.where(rc_1d>0.01)[0]
        if len(idx_ct) > 0:
            Zct = z_dim[idx_ct[-1]]
            idx_cb = np.where(rnp_1d<0.01)[0]
            Zcb = z_dim[idx_cb[0]]
        else:
            Zct = 0
            Zcb = 0

    # ccp at differnt layers
    rc_1d = rc2d[:,i]*1000 # g/kg
    ro_1d = ro2d[:zlev,i]
    ccp_L = np.trapz(ro_1d[:17]*rc_1d[:17],z_dim[:17]*1000)/(twv_path) # g/kg
    ccp_M = np.trapz(ro_1d[17:26]*rc_1d[17:26],z_dim[17:26]*1000)/(twv_path)
    ccp_U = np.trapz(ro_1d[26:34]*rc_1d[26:34],z_dim[26:34]*1000)/(twv_path)

    # Wmax 
    w_1d = w2d[:,i] # m/s
    Wmax = np.max(w_1d)
    Wmax_z = z_dim[np.where(w_1d == np.max(w_1d))[0]]

    # categorize into deep, stratiform and shallow by above defined paramters
    # initialize
    deep_region = 0
    shw_region = 0
    stra_region = 0
        
    try:
        if ((Zct > 7) and (ccp_M > 25) and (Wmax > 3) and (Zcb < 4)) or (Wmax > 6):
            deep_region = 1

        if (Zct < 7 and Zcb < 2.5 and Zcb >0) or (ccp_L > 30 and ccp_U < 15) or (Wmax_z < 3):
            if deep_region == 0: # not assigned yet
                shw_region = 1

        if (deep_region + shw_region) == 0:
            rc_1d = rc2d[:,i]*1000 # g/kg
            ro_1d = ro2d[:zlev,i] # kg/m^3
            if np.trapz(ro_1d*rc_1d,z_dim*1000) > 150: # total condensate g/m^2, used to be 10
                stra_region = 1
    except:
        deep_region = 0
        shw_region = 0
        stra_region = 0
            
    return deep_region, shw_region, stra_region

In [12]:
def cloudpartition_2D(rp2d, rnp2d, ro2d, w2d, twv_path, y_dim, x_dim, z_dim, zlev):
    "input: 3d and 2d arrays at a certain timestep"
    
    # ==== multiprocessing data columns here ====
    proccess_pool = Pool()
    df = proccess_pool.map(partition_multiprocess,range(ny*nx))
    proccess_pool.close()
    # ======== END ========
    
    deep_region = np.zeros((ny*nx))
    shw_region = np.copy(deep_region)
    stra_region = np.copy(deep_region)
    for i in range(len(df)):
        deep_region[i] = df[i][0]
        shw_region[i] = df[i][1]
        stra_region[i] = df[i][2]
 
    # post processing to double check no overlabeled pixels
    deep_region2D = np.copy(deep_region.reshape((len(y_dim),len(x_dim))))
    shw_region2D = np.copy(shw_region.reshape((len(y_dim),len(x_dim))))
    stra_region2D = np.copy(stra_region.reshape((len(y_dim),len(x_dim))))

    idx,idy = np.where(deep_region2D==1)

    for i,j in zip(idx,idy):
        if deep_region2D[i,j] == 1:
            deep_region2D[i-1:i+2,j-1:j+2] = 1
            if shw_region2D[i,j] == 1:
                shw_region2D[i,j] == 0
            if stra_region2D[i,j] == 1:
                shw_region2D[i,j] == 0

    for i in range(len(y_dim)):
        for j in range(len(x_dim)):
            total_label = deep_region2D + shw_region2D + stra_region2D

            if total_label[i,j] > 1:
                deep_region2D[i,j] = 1
                shw_region2D[i,j] = 0
                stra_region2D[i,j] = 0 
                
    return deep_region2D, shw_region2D, stra_region2D

In [13]:
%%time

timestamps = np.arange(72,0,-1) # last day stats
deep_region2D = np.zeros((len(timestamps),len(y_dim),len(x_dim)))
shw_region2D = np.copy(deep_region2D)
stra_region2D = np.copy(deep_region2D)

zlev = 45 # top level for calculation
ny = len(y_dim)
nx = len(x_dim)
ro2d = np.tile(ro1[:zlev,:].mean(axis=(1,2)),(nx,ny,1)).swapaxes(0,2).reshape((zlev,ny*nx))

for n,t in enumerate(timestamps):
    
    print(t)
    # reshape into 2d data for efficient computation 
    rc2d = (rp3d1[-t,:]+rnp3d1[-t,:]).values.reshape((zlev,ny*nx)) # 3d field of rc = r_p + r_np'
    rnp2d = rnp3d1[-t,:].values.reshape((zlev,ny*nx))
    rp2d = rp3d1[-t,:].values.reshape((zlev,ny*nx))
    w2d = w3d1[-t,:].values.reshape((zlev,ny*nx)) # 3d field of vertical velocity
    twv_path = pwat1[-t,:].mean().values # single value

    deep_region2D[n,:], shw_region2D[n,:], stra_region2D[n,:] = cloudpartition_2D(rp2d, rnp2d, ro2d,
                                                                   w2d,twv_path,
                                                                   y_dim, x_dim, 
                                                                   z_dim, zlev)

72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
CPU times: user 6min 3s, sys: 38.5 s, total: 6min 42s
Wall time: 12min 4s


In [14]:
# t=18

# rc2d = (rp3d1[-t,:]+rnp3d1[-t,:]).values.reshape((zlev,ny*nx)) # 3d field of rc = r_p + r_np'
# ro2d = np.tile(ro1[:zlev,:].mean(axis=(1,2)),(nx,ny,1)).swapaxes(0,2).reshape((zlev,ny*nx))
# rnp2d = rnp3d1[-t,:].values.reshape((zlev,ny*nx))
# rp2d = rp3d1[-t,:].values.reshape((zlev,ny*nx))
# w2d = w3d1[-t,:].values.reshape((zlev,ny*nx)) # 3d field of vertical velocity
# twv_path = pwat1[-t,:].mean().values # single value

# deep_region2D[n,:], shw_region2D[n,:], stra_region2D[n,:] = cloudpartition_2D(rp2d, rnp2d, ro2d,
#                                                                w2d,twv_path,
#                                                                y_dim, x_dim, 
#                                                                z_dim, zlev)

In [15]:
os.system('mkdir -p ./data_out') # create output for later
os.chdir(server+exp_name[0]+'/data_out')

# cloud fraction & rainfall contribution, in sheared and non-sheared regimes
s_int = deep_region2D.shape[1]//4
s_end = 3*deep_region2D.shape[1]//4
nys = 2*deep_region2D.shape[1]//4

deep_frac_s = np.sum(deep_region2D[:,s_int:s_end,:],axis=(1,2))/(nx*nys)
deep_frac_ns = np.sum(np.concatenate([deep_region2D[:,:s_int,:],deep_region2D[:,-s_int:,:]],axis=1),axis=(1,2))/(nx*nys)
deep_frac_all = np.sum(deep_region2D,axis=(1,2))/(nx*ny)

stra_frac_s = np.sum(stra_region2D[:,s_int:s_end,:],axis=(1,2))/(nx*nys)
stra_frac_ns = np.sum(np.concatenate([stra_region2D[:,:s_int,:],stra_region2D[:,-s_int:,:]],axis=1),axis=(1,2))/(nx*nys)
stra_frac_all = np.sum(stra_region2D,axis=(1,2))/(nx*ny)

#shw_frac = np.sum(shw_region2D,axis=(1,2))/(nx*ny)    
    
deep_contribute_all = (deep_region2D*prate1[-len(timestamps):,:,:]) # full domain
deep_contribute_s = deep_contribute_all[:,s_int:s_end,:] # subdomain, shear
deep_contribute_ns = np.concatenate([deep_contribute_all[:,:s_int,:],deep_contribute_all[:,-s_int:,:]],axis=1) # unshear

stra_contribute_all = (stra_region2D*prate1[-len(timestamps):,:]) # full domain
stra_contribute_s = stra_contribute_all[:,s_int:s_end,:] # subdomain, shear
stra_contribute_ns = np.concatenate([stra_contribute_all[:,:s_int,:],stra_contribute_all[:,-s_int:,:]],axis=1) # unshear

prate_all = prate1[-len(timestamps):,:,:]
prate_s = prate_all[:,s_int:s_end,:]
prate_ns = np.concatenate([prate_all[:,:s_int,:],prate_all[:,-s_int:,:]],axis=1)
    
rainfrac_deep_s = np.sum(deep_contribute_s,axis=(1,2))/np.sum(prate_s,axis=(1,2))
rainfrac_deep_ns = np.sum(deep_contribute_ns,axis=(1,2))/np.sum(prate_ns,axis=(1,2))
rainfrac_deep_all = np.sum(deep_contribute_all,axis=(1,2))/np.sum(prate_all,axis=(1,2))

rainfrac_stra_s = np.sum(stra_contribute_s,axis=(1,2))/np.sum(prate_s,axis=(1,2))
rainfrac_stra_ns = np.sum(stra_contribute_ns,axis=(1,2))/np.sum(prate_ns,axis=(1,2))
rainfrac_stra_all = np.sum(stra_contribute_all,axis=(1,2))/np.sum(prate_all,axis=(1,2))

np.save('deep_frac_s.npy',deep_frac_s)
np.save('deep_frac_ns.npy',deep_frac_ns)
np.save('deep_frac_all.npy',deep_frac_all)
np.save('stra_frac_s.npy',stra_frac_s)
np.save('stra_frac_ns.npy',stra_frac_ns)
np.save('stra_frac_all.npy',stra_frac_all)

np.save('deep_rainfrac_s.npy',rainfrac_deep_s)
np.save('deep_rainfrac_ns.npy',rainfrac_deep_ns)
np.save('deep_rainfrac_all.npy',rainfrac_deep_all)
np.save('stra_rainfrac_s.npy',rainfrac_stra_s)
np.save('stra_rainfrac_ns.npy',rainfrac_stra_ns)
np.save('stra_rainfrac_all.npy',rainfrac_stra_all)

In [ ]:
# os.system('mkdir -p ./data_out') # create output for later
# os.chdir(server+exp_name[0]+'/data_out')

# # cloud fraction & rainfall contribution
# deep_frac = np.sum(deep_region2D,axis=(1,2))/(nx*ny)
# stra_frac = np.sum(stra_region2D,axis=(1,2))/(nx*ny)
# shw_frac = np.sum(shw_region2D,axis=(1,2))/(nx*ny)    
    
# rainfrac_deep = np.sum(deep_region2D*prate1[-len(timestamps):,:],axis=(1,2))/np.sum(prate1[-len(timestamps):,:],axis=(1,2))
# rainfrac_stra = np.sum(stra_region2D*prate1[-len(timestamps):,:],axis=(1,2))/np.sum(prate1[-len(timestamps):,:],axis=(1,2))
# rainfrac_shw = np.sum(shw_region2D*prate1[-len(timestamps):,:],axis=(1,2))/np.sum(prate1[-len(timestamps):,:],axis=(1,2))


# np.save('deep_frac.npy',deep_frac)
# np.save('stra_frac.npy',stra_frac)
# np.save('shw_frac.npy',shw_frac)

# np.save('deep_rainfrac.npy',rainfrac_deep)
# np.save('stra_rainfrac.npy',rainfrac_stra)
# np.save('shw_rainfrac.npy',rainfrac_shw)

In [ ]:
# simple demonstration
fig,ax = plt.subplots(1,1,figsize=(5,6))
cf=ax.pcolormesh(x_dim,y_dim,deep_region2D[-10,:,:],vmin=0.5)
cf.cmap.set_under('whitesmoke')
ax.contour(x_dim,y_dim,prate1[-10,:,:]/24,levels=[0.5,1,3,5,10],colors=['k'],linewidths=1)
ax.set_aspect('equal')
ax.set_xlabel('x [km]')
ax.set_ylabel('y [km]')
ax.set_title('Deep Conv',fontsize=14)

In [ ]:
# simple demonstration
fig,ax = plt.subplots(1,1,figsize=(5,6))
cf=ax.pcolormesh(x_dim,y_dim,stra_region2D[-10,:,:],vmin=0.5)
cf.cmap.set_under('whitesmoke')
ax.contour(x_dim,y_dim,prate1[-10,:,:]/24,levels=[0.5,1,3,5,10],colors=['k'],linewidths=1)
ax.set_aspect('equal')
ax.set_xlabel('x [km]')
ax.set_ylabel('y [km]')
ax.set_title('Stratiform',fontsize=14)

### Diagnosis section

In [ ]:
server = '/data2/willytsai/cm1r19.8/analysis/runs_cheyenne/'
exp_name = ['CM1_RAD4K_MPI_SFXint',
            'CM1_RAD4K_RCEP_240x240_lowU05',
            'CM1_RAD4K_RCEP_240x240_lowU07',
            'CM1_RAD4K_RCEP_240x240_lowU10',
            'CM1_RAD4K_RCEP_240x240_lowU12',
            'CM1_RAD4K_RCEP_240x240_lowU15']

In [ ]:
tmp = np.load(server+exp_name[0]+'/data_out/deep_frac_all.npy')
timesteps = len(tmp)

deep_frac_all = np.zeros((len(exp_name),timesteps))
deep_frac_s = np.copy(deep_frac_all)
deep_frac_ns = np.copy(deep_frac_all)
stra_frac_all = np.copy(deep_frac_all)
stra_frac_s = np.copy(deep_frac_all)
stra_frac_ns = np.copy(deep_frac_all)

rainfrac_deep_all = np.copy(deep_frac_all)
rainfrac_deep_s = np.copy(deep_frac_all)
rainfrac_deep_ns = np.copy(deep_frac_all)

rainfrac_stra_all = np.copy(deep_frac_all)
rainfrac_stra_s = np.copy(deep_frac_all)
rainfrac_stra_ns = np.copy(deep_frac_all)

for n in range(len(exp_name)):
    deep_frac_all[n,:] = np.load(server+exp_name[n]+'/data_out/deep_frac_all.npy')
    deep_frac_s[n,:] = np.load(server+exp_name[n]+'/data_out/deep_frac_s.npy')
    deep_frac_ns[n,:] = np.load(server+exp_name[n]+'/data_out/deep_frac_ns.npy')
    
    stra_frac_all[n,:] = np.load(server+exp_name[n]+'/data_out/stra_frac_all.npy')
    stra_frac_s[n,:] = np.load(server+exp_name[n]+'/data_out/stra_frac_s.npy')
    stra_frac_ns[n,:] = np.load(server+exp_name[n]+'/data_out/stra_frac_ns.npy')
    
    rainfrac_deep_all[n,:] = np.load(server+exp_name[n]+'/data_out/deep_rainfrac_all.npy')
    rainfrac_deep_s[n,:] = np.load(server+exp_name[n]+'/data_out/deep_rainfrac_s.npy')
    rainfrac_deep_ns[n,:] = np.load(server+exp_name[n]+'/data_out/deep_rainfrac_ns.npy')

    rainfrac_stra_all[n,:] = np.load(server+exp_name[n]+'/data_out/stra_rainfrac_all.npy')
    rainfrac_stra_s[n,:] = np.load(server+exp_name[n]+'/data_out/stra_rainfrac_s.npy')
    rainfrac_stra_ns[n,:] = np.load(server+exp_name[n]+'/data_out/stra_rainfrac_ns.npy')


In [ ]:
fig,ax = plt.subplots(1,2,figsize=(8,4))

colors = ['k','r','g','b','m','darkred']
exp_short = ['CTRL','U05','U07','U10','U12','U15']

for n in range(len(exp_name)):
    deep_mean_s = deep_frac_s[n,:].mean()
    stra_mean_s = stra_frac_s[n,:].mean()
    raindeep_mean_s = rainfrac_deep_s[n,:].mean()
    rainstra_mean_s = rainfrac_stra_s[n,:].mean()
    
    deep_mean_ns = deep_frac_ns[n,:].mean()
    stra_mean_ns = stra_frac_ns[n,:].mean()
    raindeep_mean_ns = rainfrac_deep_ns[n,:].mean()
    rainstra_mean_ns = rainfrac_stra_ns[n,:].mean()
    
    # reference
    dfs_ref = deep_frac_s[0,:].mean()
    sts_ref = stra_frac_s[0,:].mean()
    rds_ref = rainfrac_deep_s[0,:].mean()
    rss_ref = rainfrac_stra_s[0,:].mean()
    
    # shear region
    ax[0].plot([1,2],[deep_mean_s,stra_mean_s],'^',markersize=10,color=colors[n],linewidth=2)
    ax[1].plot([1,2],[raindeep_mean_s,rainstra_mean_s],'^',markersize=10,color=colors[n],linewidth=2)
    # unshear region
    #ax[0].plot([1,2],[deep_mean_ns,stra_mean_ns],'o',markersize=10,color=colors[n])
    #ax[1].plot([1,2],[raindeep_mean_ns,rainstra_mean_ns],'o',markersize=10,color=colors[n])
    
ax[0].legend(['CTRL','U05','U07','U10','U12','U15'])
ax[0].set_ylabel('Fraction',fontsize=14)
ax[0].set_xticks([1,2]);ax[0].set_xlim([0.5,2.5]);ax[0].grid(axis='y',linestyle=':')
ax[0].set_xticklabels(['Deep Conv.','Stratiform'])

ax[1].set_ylabel('Rainfall contribution',fontsize=14)
ax[1].set_xticks([1,2]);ax[1].set_xlim([0.5,2.5]);ax[1].grid(axis='y',linestyle=':')
ax[1].set_xticklabels(['Deep Conv.','Stratiform'])
plt.tight_layout()
#ax[0].set_ylim([0,1])
#ax[1].set_ylim([0,1])

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(8,5))

colors = ['k','r','g','b','m','darkred']
exp_short = ['CTRL','U05','U07','U10','U12','U15']

for n in range(len(exp_name)):
    deep_mean_s = deep_frac_s[n,:].mean()
    stra_mean_s = stra_frac_s[n,:].mean()
    raindeep_mean_s = rainfrac_deep_s[n,:].mean()
    rainstra_mean_s = rainfrac_stra_s[n,:].mean()
    
    deep_mean_ns = deep_frac_ns[n,:].mean()
    stra_mean_ns = stra_frac_ns[n,:].mean()
    raindeep_mean_ns = rainfrac_deep_ns[n,:].mean()
    rainstra_mean_ns = rainfrac_stra_ns[n,:].mean()
    
    # shear region
    ax[0].plot([1,2],[deep_mean_s,stra_mean_s],'^',markersize=10,color=colors[n])
    ax[1].plot([1,2],[raindeep_mean_s,rainstra_mean_s],'^',markersize=10,color=colors[n])
    # unshear region
    ax[0].plot([1,2],[deep_mean_ns,stra_mean_ns],'o',markersize=10,color=colors[n])
    ax[1].plot([1,2],[raindeep_mean_ns,rainstra_mean_ns],'o',markersize=10,color=colors[n])
    
ax[0].legend(['S','US'])
ax[0].set_ylabel('Fraction',fontsize=13)
ax[0].set_xticks([1,2]);ax[0].set_xlim([0.5,2.5]);ax[0].grid(axis='y',linestyle=':')
ax[0].set_xticklabels(['Deep Conv.','Stratiform'])

ax[1].set_ylabel('Rain contribution',fontsize=13)
ax[1].set_xticks([1,2]);ax[1].set_xlim([0.5,2.5]);ax[1].grid(axis='y',linestyle=':')
ax[1].set_xticklabels(['Deep Conv.','Stratiform'])
#ax[0].set_ylim([0,1])
#ax[1].set_ylim([0,1])

In [ ]:
import pandas as pd

In [ ]:

# managing the bar chart by pandas 
df = pd.DataFrame([['Deep Conv.', 10, 20, 10, 30], ['Stratiform', 20, 25, 15, 25], ['C', 12, 15, 19, 6],
                   ['D', 10, 29, 13, 19]],)

deep_frac_list = [x for x in deep_frac_s.mean(axis=1)]

plt.bar(['Deep Conv.'],deep_frac_list,
       width=0.35)